In [1]:
from cosmos_rl.policy.model.qwen2_5_vl import Qwen2_5_VLConditionalModel
from transformers import Qwen2_5_VLForConditionalGeneration
import torch

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 07-23 22:22:10 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 07-23 22:22:10 [__init__.py:239] Automatically detected platform cuda.


2025-07-23 22:22:12,232	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
device = torch.device("cuda")
hf_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
).to(device)

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 40.44it/s]


In [3]:
cosmos_model = Qwen2_5_VLConditionalModel.from_pretrained(
    hf_model.config,
    "Qwen/Qwen2.5-VL-7B-Instruct"
)

[cosmos] 2025-07-23 22:22:28,954 - cosmos - INFO - model path Qwen/Qwen2.5-VL-7B-Instruct is not a directory. Trying to load from HuggingFace Hub...
[cosmos] 2025-07-23 22:22:29,242 - cosmos - INFO - Found safetensors in Qwen/Qwen2.5-VL-7B-Instruct. Ignoring *pytorch_model* and *consolidated* files.
Fetching 1 files: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 5849.80it/s]
[cosmos] 2025-07-23 22:22:29,439 - cosmos - INFO - Downloaded model from HuggingFace to /root/.cache/huggingface/transformers/models--Qwen--Qwen2.5-VL-7B-Instruct/snapshots/cc594898137f460bfe9f0759e9844b3ce807cfb5/model.safetensors.index.json
[cosmos] 2025-07-23 22:22:29,440 - cosmos - INFO - model path Qwen/Qwen2.5-VL-7B-Instruct is not a directory. Trying to load from HuggingFace Hub...
[cosmos] 2025-07-23 22:22:29,440 - cosmos - INFO - Found safetensors in Qwen/Qwen2.5-VL-7B-Instruct. Ignoring *pytorch_model* and *consolidated* files.
Fetching

In [4]:
cosmos_model = cosmos_model.to(device).to(torch.bfloat16)


In [5]:
from cosmos_rl.policy.config import ParallelismConfig
from cosmos_rl.utils.parallelism import ParallelDims

cosmos_model.load_hf_weights(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    parallel_dims=ParallelDims.from_config(
        ParallelismConfig(
            tp_size=1,
            pp_size=1,
            dp_replicate_size=1,
            dp_shard_size=1,
        )
    ),
    device=device,
)

cosmos_model.model.rotary_emb.inv_freq = cosmos_model.model.rotary_emb.inv_freq.to(torch.float32)
cosmos_model.model.rotary_emb.reset_inv_freq()
cosmos_model.visual.rotary_pos_emb.inv_freq = cosmos_model.visual.rotary_pos_emb.inv_freq.to(torch.float32)
cosmos_model.visual.rotary_pos_emb.reset_inv_freq()

[cosmos] 2025-07-23 22:23:01,184 - cosmos - INFO - model path Qwen/Qwen2.5-VL-7B-Instruct is not a directory. Trying to load from HuggingFace Hub...
[cosmos] 2025-07-23 22:23:01,185 - cosmos - INFO - Found safetensors in Qwen/Qwen2.5-VL-7B-Instruct. Ignoring *pytorch_model* and *consolidated* files.
Fetching 16 files: 100%|██████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 157532.54it/s]
[cosmos] 2025-07-23 22:23:01,339 - cosmos - INFO - Downloaded model from HuggingFace to /root/.cache/huggingface/transformers/models--Qwen--Qwen2.5-VL-7B-Instruct/snapshots/cc594898137f460bfe9f0759e9844b3ce807cfb5


In [6]:
from qwen_vl_utils import process_vision_info
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
                "max_pixels": 89100,
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(device)
input_ids = inputs.input_ids
image_grid_thw = inputs.image_grid_thw
pixel_values_lengths_per_sample = torch.tensor(inputs.pixel_values.shape[0]).to(device).int()


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [7]:
cosmos_model.post_to_empty_hook(None)
position_ids, input_ids, seq_dim_idx = cosmos_model.get_position_ids(
    **inputs
)
with torch.no_grad():
    cosmos_result = cosmos_model.visual.forward(
        inputs.pixel_values.unsqueeze(0).to(torch.bfloat16).to(device),
        grid_thw=image_grid_thw.to(device),
    )
    print(cosmos_result)

tensor([[-0.7734, -0.1328,  0.2949,  ..., -0.9688, -0.1348, -1.0156],
        [-1.0547,  0.1562, -0.7422,  ..., -0.2441, -0.1006, -1.1406],
        [-0.1016,  1.3906,  0.3730,  ..., -1.1016,  0.1699, -0.9727],
        ...,
        [-1.2578,  1.2656,  0.7695,  ...,  1.4062, -1.0000, -2.0000],
        [-0.6055,  0.3086,  0.6641,  ..., -1.3438, -0.8984, -0.3516],
        [-0.2490,  0.0044, -0.1475,  ..., -0.1826,  0.0996, -0.2373]],
       device='cuda:0', dtype=torch.bfloat16)


In [8]:
with torch.no_grad():
    hf_result = hf_model.visual(
        inputs.pixel_values.unsqueeze(0).unsqueeze(0),
        inputs.image_grid_thw.to(device),
    )
    print(hf_result)

tensor([[-7.6953e-01, -1.2695e-01,  2.8906e-01,  ..., -9.7656e-01,
         -1.3477e-01, -1.0078e+00],
        [-1.0625e+00,  1.7188e-01, -7.4219e-01,  ..., -2.4609e-01,
         -1.1865e-01, -1.1484e+00],
        [-1.1182e-01,  1.3906e+00,  3.7500e-01,  ..., -1.0938e+00,
          1.7480e-01, -9.7266e-01],
        ...,
        [-1.2734e+00,  1.2578e+00,  7.6953e-01,  ...,  1.3750e+00,
         -1.0156e+00, -2.0156e+00],
        [-6.3281e-01,  2.9492e-01,  6.6797e-01,  ..., -1.3516e+00,
         -8.7109e-01, -3.5352e-01],
        [-2.5195e-01, -1.1292e-03, -1.4648e-01,  ..., -1.8359e-01,
          9.6680e-02, -2.4414e-01]], device='cuda:0', dtype=torch.bfloat16)


-----------------

In [9]:
cosmos_model.post_to_empty_hook(None)
position_ids, input_ids, seq_dim_idx = cosmos_model.get_position_ids(
    **inputs
)

with torch.no_grad():
    cosmos_result = cosmos_model.forward(
        input_ids=input_ids,
        pixel_values=inputs.pixel_values.unsqueeze(0).to(torch.bfloat16).to(device),
        image_grid_thw=image_grid_thw.to(device),
        position_ids=position_ids.to(device),
        pixel_values_lengths_per_sample=pixel_values_lengths_per_sample.view(-1, 1).to(device),
    )

In [10]:
with torch.no_grad():
    hf_result = hf_model.forward(
        **inputs
    )


In [11]:
cosmos_result[-1, -1, :]

tensor([10.1250, 14.8750, 11.4375,  ...,  1.2500,  1.2500,  1.2500],
       device='cuda:0', dtype=torch.bfloat16)

In [12]:
hf_result.logits[-1, -1, :]

tensor([10.1875, 14.8750, 11.4375,  ...,  1.1875,  1.1875,  1.1875],
       device='cuda:0', dtype=torch.bfloat16)

tensor([10.1875, 14.8750, 11.4375,  ...,  1.1875,  1.1875,  1.1875],
       device='cuda:0', dtype=torch.bfloat16)

In [ ]:
hf_result.logits[-1, -1, :]